# スクレイピングとは

ウェブページに記載されている情報の中から，必要部分だけを自動抽出すること。Web スクレイピング。Web スクレーピング。

## スクレイピングの流れ

1. スクレイピングが禁止されていないか確認する
1. 小さく情報取得してみる
    1. `requests`でHTMLを取得する
    1. 取得したいHTMLを解析する(`BeautifulSoup`)
    1. 欲しい情報を取得する
1. forループする
1. 出力する

## 基本構文

- `requests.get('URL')`でHTMLを取得する
- `find_all(タグ名,class_='クラス名')`で全てのタグ情報をクラス付きで指定する。

# suumoの情報を小さく取得する

## `requests`でURLにアクセスする

ライブラリをインポートする。

In [1]:
from bs4 import BeautifulSoup
import requests

URLを指定する。

In [3]:
url = 'https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}'
target_url = url.format(1)
print(target_url)

https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=1


In [3]:
r = requests.get(target_url)
soup = BeautifulSoup(r.text)

## `soup`から情報を抽出する

`find_all(タグ名,class_='')`で全てのタグ情報をクラス付きで指定する。

In [4]:
#cassetteクラスを持ったdivタグをすべて取得
contents = soup.find_all('div',class_='cassetteitem')

`find_all()`で取得した情報はリスト形式。`len()`で要素数を確認する。

In [5]:
len(contents)

20

`content`に最初の要素を格納する。

In [6]:
content = contents[0]

以下の項目を取得する。

- 物件情報
    - 物件名
    - 住所
    - アクセス
    - 築年数
- 部屋情報
    - 階数
    - 賃料・管理費
    - 敷金・礼金
    - 間取り・面積

### 物件情報・部屋情報が入ったブロックを取得する

In [7]:
#物件情報を変数detailに格納する
detail = content.find('div',class_='cassetteitem-detail')
#物件情報を変数detailに格納する
table = content.find('table',class_='cassetteitem_other')

### 物件情報を抽出する

In [8]:
#物件名を変数titleに格納する
title = detail.find('div',class_='cassetteitem_content-title').text
#住所を変数addressに格納する
address = detail.find('li',class_='cassetteitem_detail-col1').text
#アクセスを変数accessに格納する
access = detail.find('li',class_='cassetteitem_detail-col2').text
#築年数を変数ageに格納する
age = detail.find('li',class_='cassetteitem_detail-col3').text

中身を確認する。

In [9]:
title,address,access,age

('西新宿戸建',
 '東京都新宿区西新宿４',
 '\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初台駅 歩12分\n',
 '\n築26年\n地下1地上3階建\n')

### 部屋情報を抽出する

`<tr>`はtable rowの略で`<table>`タグの行（横一列）のこと。複数の`<tr>`タグを抽出し、ここでは最初の1つを取得する。

In [10]:
#全ての<tr>タグを変数tr_tagsに格納する
tr_tags = table.find_all('tr',class_='js-cassette_link')
#tr_tagsの中から最初の1つだけtr_tagに格納する
tr_tag = tr_tags[0]

`<tr>`は9つの`<td>`で構成されている。欲しい部屋情報は3〜6番目の`<td>`に入っている。

~~~python
#unpackと呼ばれる操作

list = [1, 2, 3, 4, 5]
a, b = list[2:4]
print(a, b)

>>> 3, 4
~~~

In [11]:
#floor、price、first_fee、capacityに4つの情報を格納する
floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

中身を確認する。

In [12]:
floor,price,first_fee,capacity

(<td>
 											1-3階</td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">48万円</span></span></li>
 <li><span class="cassetteitem_price cassetteitem_price--administration">-</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_price cassetteitem_price--deposit">48万円</span></li>
 <li><span class="cassetteitem_price cassetteitem_price--gratuity">24万円</span></li>
 </ul>
 </td>,
 <td>
 <ul>
 <li><span class="cassetteitem_madori">7LDK</span></li>
 <li><span class="cassetteitem_menseki">130.47m<sup>2</sup></span></li>
 </ul>
 </td>)

それぞれの変数に分割する。

In [13]:
#賃料と管理費を変数feeとmanagement_feeに格納する
fee,management_fee = price.find_all('li')
#敷金と礼金を変数depositとgratuityに格納する
deposit,gratuity = first_fee.find_all('li')
#間取りと面積を変数floor_planとareaに格納する
floor_plan,area = capacity.find_all('li')

中身を確認する。

In [14]:
print(floor)
print(fee)
print(management_fee)
print(deposit)
print(gratuity)
print(floor_plan)
print(area)

<td>
											1-3階</td>
<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">48万円</span></span></li>
<li><span class="cassetteitem_price cassetteitem_price--administration">-</span></li>
<li><span class="cassetteitem_price cassetteitem_price--deposit">48万円</span></li>
<li><span class="cassetteitem_price cassetteitem_price--gratuity">24万円</span></li>
<li><span class="cassetteitem_madori">7LDK</span></li>
<li><span class="cassetteitem_menseki">130.47m<sup>2</sup></span></li>


## 情報を辞書に格納する

変数dに、11項目を格納する。必要に応じて`.text`をつける。

In [15]:
d = {
    'title':title,
    'address':address,
    'access':access,
    'age':age,
    'floor':floor.text,
    'fee':fee.text,
    'management_fee':management_fee.text,
    'deposit':deposit.text,
    'gratuity':gratuity.text,
    'floor_plan':floor_plan.text,
    'area':area.text
}

中身を確認する。

In [16]:
d

{'title': '西新宿戸建',
 'address': '東京都新宿区西新宿４',
 'access': '\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初台駅 歩12分\n',
 'age': '\n築26年\n地下1地上3階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1-3階',
 'fee': '48万円',
 'management_fee': '-',
 'deposit': '48万円',
 'gratuity': '24万円',
 'floor_plan': '7LDK',
 'area': '130.47m2'}

# 1ページ目の全ての情報を取得する

In [17]:
#変数d_listに空のリストを作成する
d_list = []

#cassetteクラスを持ったdivタグをすべて取得
contents = soup.find_all('div',class_='cassetteitem')

#全ての物件情報をforループで取得する
for content in contents:
    
    #物件情報と部屋情報を取得する
    detail = content.find('div',class_='cassetteitem-detail')
    table = content.find('table',class_='cassetteitem_other')
    
    #物件情報から必要な情報を取得する
    title = detail.find('div',class_='cassetteitem_content-title').text
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #部屋情報のブロックから、各部屋情報を取得する
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    
    #各部屋情報をforループで取得する
    for tr_tag in tr_tags:
        
        #欲しい情報を取得する
        floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        
        #さらに細かい情報を取得する
        fee,management_fee = price.find_all('li')
        deposit,gratuity = first_fee.find_all('li')
        floor_plan,area = capacity.find_all('li')
        
        #取得した全ての情報を辞書に格納する
        d = {
            'title':title,
            'address':address,
            'access':access,
            'age':age,
            'floor':floor.text,
            'fee':fee.text,
            'management_fee':management_fee.text,
            'deposit':deposit.text,
            'gratuity':gratuity.text,
            'floor_plan':floor_plan.text,
            'area':area.text
        }
        
        #取得した辞書をd_listに格納する
        d_list.append(d)

中身の最初の2件を確認する。体裁を綺麗にしたまま表示できる`pprint()`を使うと便利。

In [18]:
from pprint import pprint
pprint(d_list[0])
print() #空白一行を作る
pprint(d_list[1])

{'access': '\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初台駅 歩12分\n',
 'address': '東京都新宿区西新宿４',
 'age': '\n築26年\n地下1地上3階建\n',
 'area': '130.47m2',
 'deposit': '48万円',
 'fee': '48万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t1-3階',
 'floor_plan': '7LDK',
 'gratuity': '24万円',
 'management_fee': '-',
 'title': '西新宿戸建'}

{'access': '\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初台駅 歩12分\n',
 'address': '東京都新宿区西新宿４',
 'age': '\n築26年\n地下1地上3階建\n',
 'area': '130.47m2',
 'deposit': '48万円',
 'fee': '48万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t-',
 'floor_plan': '7LDK',
 'gratuity': '24万円',
 'management_fee': '-',
 'title': '西新宿戸建'}


# 複数ページの全ての情報を取得する

複数ページにアクセスするときはサーバーに負荷をかけすぎないようにするために、処理を一時停止する。<br>`sleep(秒数)`で指定した秒数だけ休止する。

~~~python
from time import sleep
~~~

改めて頭から記述する。

In [19]:
#ライブラリのインポート
from time import sleep
from bs4 import BeautifulSoup
import requests

#空のリストを作成する
d_list = []

#URLにアクセスし、HTMLを解析する
url = 'https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}'
#1ページ目から3ページ目まで
for i in range(1, 4):
    print('d_listの大きさ:',len(d_list))
    target_url = url.format(i)
    print(target_url)
    r = requests.get(target_url)
    #sleepで一時停止する
    sleep(1)
    soup = BeautifulSoup(r.text)
    
    #cassetteクラスを持ったdivタグをすべて取得
    contents = soup.find_all('div',class_='cassetteitem')

    #全ての物件情報をforループで取得する
    for content in contents:
    
        #物件情報と部屋情報を取得する
        detail = content.find('div',class_='cassetteitem-detail')
        table = content.find('table',class_='cassetteitem_other')
        
        #物件情報から必要な情報を取得する
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text
        access = detail.find('li',class_='cassetteitem_detail-col2').text
        age = detail.find('li',class_='cassetteitem_detail-col3').text
        
        #部屋情報のブロックから、各部屋情報を取得する
        tr_tags = table.find_all('tr',class_='js-cassette_link')
        
        #各部屋情報をforループで取得する
        for tr_tag in tr_tags:
        
            #欲しい情報を取得する
            floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        
            #さらに細かい情報を取得する
            fee,management_fee = price.find_all('li')
            deposit,gratuity = first_fee.find_all('li')
            floor_plan,area = capacity.find_all('li')
            
            #取得した全ての情報を辞書に格納する
            d = {
                'title':title,
                'address':address,
                'access':access,
                'age':age,
                'floor':floor.text,
                'fee':fee.text,
                'management_fee':management_fee.text,
                'deposit':deposit.text,
                'gratuity':gratuity.text,
                'floor_plan':floor_plan.text,
                'area':area.text
            }
        
            #取得した辞書をd_listに格納する
            d_list.append(d)

d_listの大きさ: 0
https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=1
d_listの大きさ: 227
https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=2
d_listの大きさ: 381
https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=3


最後の一件の内容を確認する。

In [20]:
d_list[-1]

{'title': 'スカイコート早稲田第２',
 'address': '東京都新宿区若松町',
 'access': '\n都営大江戸線/若松河田駅 歩5分\n東京メトロ東西線/早稲田駅 歩12分\n都営大江戸線/東新宿駅 歩10分\n',
 'age': '\n築24年\n9階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t4階',
 'fee': '7万円',
 'management_fee': '10000円',
 'deposit': '-',
 'gratuity': '7万円',
 'floor_plan': 'ワンルーム',
 'area': '18m2'}

# CSVファイルに出力する

- 取得したデータを表形式にする
- 表形式のデータをCSVに出力する

## データを表形式にする

`pandas`でデータフレームに変換する

In [21]:
import pandas as pd
df = pd.DataFrame(d_list)
df.head() #先頭5行を表示する

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,floor_plan,area
0,西新宿戸建,東京都新宿区西新宿４,\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初...,\n築26年\n地下1地上3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t1-3階,48万円,-,48万円,24万円,7LDK,130.47m2
1,西新宿戸建,東京都新宿区西新宿４,\n都営大江戸線/西新宿五丁目駅 歩6分\n都営大江戸線/都庁前駅 歩12分\n京王新線/初...,\n築26年\n地下1地上3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t-,48万円,-,48万円,24万円,7LDK,130.47m2
2,パークスフィア牛込神楽坂,東京都新宿区南山伏町,\n都営大江戸線/牛込柳町駅 歩4分\n東京メトロ東西線/神楽坂駅 歩10分\n都営新宿線/...,\n新築\n地下1地上13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19.7万円,12000円,19.7万円,-,1SLDK,40.01m2
3,パークスフィア牛込神楽坂,東京都新宿区南山伏町,\n都営大江戸線/牛込柳町駅 歩4分\n東京メトロ東西線/神楽坂駅 歩10分\n都営新宿線/...,\n新築\n地下1地上13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19.7万円,12000円,19.7万円,-,1LDK,40.01m2
4,パークスフィア牛込神楽坂,東京都新宿区南山伏町,\n都営大江戸線/牛込柳町駅 歩4分\n東京メトロ東西線/神楽坂駅 歩10分\n都営新宿線/...,\n新築\n地下1地上13階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19.7万円,12000円,19.7万円,-,1LDK,40.01m2


In [22]:
df.shape #dfの大きさを確認する

(501, 11)

In [23]:
#物件名の重複を削除して、大きさを確認する
len(df.title.unique())

60

## 表形式の取得結果をCSVに出力する

`df.to_csv('出力するファイル名')`でCSV出力する

In [24]:
df.to_csv('スクレイピング.csv',index=None,encoding='utf-8-sig') #indexは上の表の左の数字列